In [1]:
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

import re
from string import punctuation
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
path_corpus_tut = Path('data/corpora/tutby_126784.csv')
path_corpus_tut_clean = Path('data/corpora/clean_tutby_126784.csv')

In [3]:
%%time

data = pd.read_csv(path_corpus_tut)

corpus = data['document']

print(data.shape)
display(data.head(3))

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,['министерства']


Wall time: 9 s


In [4]:
russian_stop_words = set(stopwords.words('russian'))
mystem = Mystem() 

def preprocess_text(text):
    text = text.lower()
    text = re.sub(u'\xa0|\n', ' ', text)
    text = re.sub('[^а-яa-z ]', '', text)
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if ((token not in russian_stop_words) and (token.strip() not in punctuation) and (len(token) > 2))]
    text = ' '.join(tokens)
    return text

In [5]:
%%time

separator = 'qsefthukoaxdvgnjthukoaxdvefcngwojd'
batch_size = 1000

result = []

for i in tqdm(range(len(corpus) // batch_size + 1)):
    batch = corpus[i * batch_size:(i + 1) * batch_size]
    batch_joint = f' {separator} '.join(batch)
    batch_clean = preprocess_text(batch_joint)
    batch_clean = batch_clean.split(separator)
    result += batch_clean
    
result = [_.strip() for _ in result]


print(len(corpus), len(result))
print(corpus.iloc[-77], result[-77])

data['document'] = result
data.to_csv(path_corpus_tut_clean, index=False)


126784 126784
Ora — один из дочерних брендов китайской компании Great Wall, который был создан в 2018 году для выпуска электромобилей. В модельном ряду Ora уже есть три машины — лифтбек iQ, а также парочка городских хэтчбеков R1 и R2, а теперь на подходе и четвертая модель, прежде известная под индексом ES11.



Теперь у электромобиля, во внешности которого можно найти сходство и с моделями Porsche, появилось имя собственное — с китайского оно переводится как Good Cat (то есть «хорошая кошка»). По габаритам новинка, для которой доступны двухцветные варианты окраски кузова, оказывается заметно крупнее R1 и R2 — как сообщают китайские СМИ, в длину Good Cat насчитывает 4,23 м, в ширину 1,82 м, а колесная база достигает 2,65 м.



В салоне в отделке которого широко используется искусственная кожа, могут разместиться до пяти седоков, из прочих особенностей — двухспицевый руль и единая панель с экранами приборов и мультимедиасистемы.



В движение Good Cat приводит установленный на передней

In [6]:
data = pd.read_csv(path_corpus_tut_clean)
print(data.shape)
display(data.head())

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства']
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,чемпион беларусь бате воспользоваться пауза че...,"['футбол', 'БАТЭ']"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,генпрокуратура украина открывать уголовный про...,"['Ситуация в Украине', 'государственные перево..."
